<a href="https://colab.research.google.com/github/T2718/AI_Make_Image/blob/main/AI4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install diffusers transformers accelerate

import os
import torch
from diffusers import StableDiffusionPipeline
from PIL import Image
import requests
from io import BytesIO

# Stable Diffusionのモデルをロード
model_id = "stablediffusionapi/eleet-model"
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    safety_checker=None  # Safe checkを無効に
)
pipe = pipe.to("cuda")  # GPUが必要です

# 画像を保存するディレクトリの作成
output_dir = "generated_images"
os.makedirs(output_dir, exist_ok=True)

# ユーザーからの入力を取得
prompt = input("プロンプトを入力してください (強さをつける場合は形式: 'prompt1:weight, prompt2:weight'): ")
negative_prompt = input("ネガティブプロンプトを入力してください: ")
num_images = int(input("生成する画像の枚数を入力してください: "))
num_inference_steps = int(input("推論ステップ数を入力してください: "))
mode = input("text to image または image to image を選んでください (t/i): ")

# プロンプトをファイル名用にサニタイズ
sanitized_prompt = "".join(c for c in prompt if c.isalnum() or c in (' ', '_')).rstrip()

# 画像生成
images = []
if mode.lower() == 't':
    for i in range(num_images):
        image = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=num_inference_steps
        ).images[0]
        images.append(image)
        # 画像をファイルに保存 (プロンプトに基づくファイル名)
        image.save(os.path.join(output_dir, f"{sanitized_prompt}_text_to_image_{i + 1}.png"))
elif mode.lower() == 'i':
    image_url = input("元画像のURLを入力してください: ")
    response = requests.get(image_url)
    original_image = Image.open(BytesIO(response.content)).convert("RGB")

    for i in range(num_images):
        image = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            image=original_image,
            num_inference_steps=num_inference_steps
        ).images[0]
        images.append(image)
        # 画像をファイルに保存 (プロンプトに基づくファイル名)
        image.save(os.path.join(output_dir, f"{sanitized_prompt}_image_to_image_{i + 1}.png"))

print(f"生成された画像は '{output_dir}' フォルダに保存されました。")
